In [1]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import glob
import re
import urllib.request
from urllib.parse import quote
from urllib.parse import unquote

r = re.compile("[а-яА-Я0-9]+")
def isInt(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
def isFloat(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

count=0
final=[]
whole_text=[]
for filename in glob.glob('rucontext\*.txt'):
    count+=1
    Is_texts=False
    toAppend=[]
    with open(filename,encoding='utf-8') as f:
        prev_sentence=''
        main_word=unquote(filename)
        main_word=main_word.replace('rucontext\\','')
        main_word=main_word.replace('.txt','')
        for line in f:
            russian = [w for w in filter(r.match, line.split())]
            if len(russian)==0:
                continue
            strok=''
            for word in russian:
                strok+=word+' '
            if not isInt(strok) and isFloat(prev_sentence) and not Is_texts:
                Is_texts=True
            if Is_texts and isInt(strok) and not isFloat(prev_sentence):
                toAppend.append([prev_sentence,strok,main_word])
            prev_sentence=strok
    final.append(toAppend)

In [58]:
import glob
symbols=[]
for filename in glob.glob('Symbols\*.txt'):
    line=''
    filename=filename.replace('Symbols\\','')
    filename=filename.replace('.txt','')
    for word in filename.split('_'):
        line+=word+' '
    tokens = prepare_text_for_lda(line)
    for token in tokens:
        symbols.append(token)
with open('symbols.txt','w',encoding='utf-8') as f:
    for symbol in symbols:
        f.write('{} \n'.format(symbol))
print(symbols)
print('ready')

['ёкухайнный', 'ёнван', 'ёндон', 'ёндын', 'мама', 'ёно-рана', 'сео-нё', 'ёрмунганд', 'ётун', 'а-невать', 'аарон', 'аарон', 'поддерживать', 'рука', 'моисей', 'аарра', 'абак', 'абант', 'мелампа', 'абант', 'посейдон', 'абант', 'царь', 'аргоса', 'абарид', 'абас', 'аббат', 'аббатисса', 'геральд', 'аббатский', 'геральд', 'аббревиатура', 'геральд', 'абгаль', 'абдала', 'абдёр', 'абдикация', 'абзар', 'ияс', 'абнауать', 'абориген', 'аботенить', 'абракадабра', 'абраксас', 'абрикос', 'абрикос', 'геральд', 'абрикосовый', 'дерево', 'геральд', 'абрскил', 'абунданция', 'аваддон', 'аваллон', 'авалокитешвар', 'аватар', 'авгия', 'авгур', 'август', 'месяц', 'август', 'цезарь', 'августин', 'гиппонский', 'дзуар', 'авель', 'авернский', 'озеро', 'авигея', 'авимелех', 'авонавилон', 'авраам', 'авраг', 'могойя', 'аврор', 'аврор', 'цвета', 'геральд', 'авсеня', 'авсон', 'австралийский', 'мифология', 'автолик', 'автомедонт', 'автомобиль', 'агава', 'агальматолит', 'агамед', 'агамемнон', 'агарь', 'агастие', 'агасфера


ready


In [4]:
import pymorphy2
import csv
 
FILENAME = "rucsv\\"
def neighbor(pos1,pos2):
    if (pos1=="NOUN") and ((pos2=="VERB")or(pos2=="ADJF")):
        return True
    if ((pos1=="VERB") or (pos1=="ADJF")) and (pos2=="NOUN"):
        return True
    return False

Special_Dict=[{'агония':2},{'ад':2},{'ад':3},
              {'азбука':2},{'акула':2},{'алмаз':3},
              {'алтарь':3},{'аромат':2},{'артерия':2},
              {'архитектор':2},{'атмосфера':4},{'атом':2},{'ангел':3},
             {'багаж':3},{'балласт':3},{'бальзам':2},{'банда':2},{'басня':2},
             {'бездна':3},{'бездна':4},{'битва':4},{'бич':2},{'бог':4},
              {'бремя':2},{'букашка':2},{'буря':2},{'буря':3}]
AddDict=[]
morph = pymorphy2.MorphAnalyzer()
filename=''
for something in final:
    AddDict=[]
    for obj in something:
        arg1=''
        arg2=''
        label=0
        tokens=prepare_text_for_lda(obj[0])
        for i in range(len(tokens)):
            p=morph.parse(tokens[i])[0]
            if p.normal_form==obj[2]:
                if len(tokens)<2:
                    continue
                if i==0:
                    neigh=morph.parse(tokens[i+1])[0]
                    if neighbor(p.tag.POS,neigh.tag.POS):
                        arg1=neigh.normal_form
                elif i==len(tokens)-1:
                    neigh=morph.parse(tokens[i-1])[0]
                    if neighbor(p.tag.POS,neigh.tag.POS):
                        arg1=neigh.normal_form
                else:
                    neigh1=morph.parse(tokens[i-1])[0]
                    neigh2=morph.parse(tokens[i+1])[0]
                    if neighbor(p.tag.POS,neigh1.tag.POS):
                        arg1=neigh1.normal_form
                    if neighbor(p.tag.POS,neigh2.tag.POS):
                        arg2=neigh2.normal_form
        for pair in Special_Dict:
            if obj[2] in pair:
                if int(pair[obj[2]])==int(obj[1]):
                    label=1
                    break
        AddDict.append({"arg1":arg1,"arg2":arg2,"word":obj[2],"sentence":obj[0],"verb_idx":obj[1],"label":label})
        filename=obj[2]
    with open(FILENAME+filename+'.csv', "w", newline="") as file:
        columns = ["arg1","arg2","word","sentence","verb_idx","label"]
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writeheader()
        for record in AddDict:
            try:
                writer.writerow(record)
            except UnicodeEncodeError as e:
                continue
    
    
'''
            for symbol in symbols:
                if p.normal_form in symbol:
                        label=1
'''

'\n            for symbol in symbols:\n                if p.normal_form in symbol:\n                        label=1\n'

In [91]:
import csv
 
FILENAME = "rucsv\\"
with open(FILENAME+filename+'.csv', "w", newline="") as file:
    columns = ["arg1","arg2","word","sentence","verb_idx","label"]
    writer = csv.DictWriter(file, fieldnames=columns)
    writer.writeheader()
    for record in AddDict:
        try:
            writer.writerow(record)
        except UnicodeEncodeError as e:
            continue
            
    #writer.writerows(AddDict)

In [85]:
for sentence in final[1]:
    print(sentence)

['1 900 65 1860 2 литровые банки 1 чайный стакан Для приготовления на зиму компотов более всего пригодны вишня черешня сливы абрикосы персики крыжовник смородина и клубника Плоды обмыть отцедить перебрать удалить веточки ', '1 ', 'абрикос']
['1 Абрикосов Хрисанф Николаевич 1877 1957 единомышленник Л Н Толстого был женат на его внучатой племяннице Н Л Оболенской С осени 1902 г по 19 июня 1903 г подолгу жил в Ясной Поляне и помогал писателю в переписке его произведений и в ответах на письма ', '1 ', 'абрикос']
['1 но не это было важно а важно приподнятое настроение которое мы застали в городе и в гостинице что к моему утешению выразилось между прочим в том что в каждом номере гостиницы стояла большая ваза полная бесплатных фруктов абрикосов персиков груш слив и винограда так что с голоду нельзя было умереть 1 ', '1 ', 'абрикос']
['2004.04 06 20 14 в Хиппе есть абрикос с творогом наверное еще какие-то но мы только этот пробовали собираюсь набрать с собой на отдых там не будет доступа к мо

['Важно только помнить что фруктовые пюре помимо вкусовых и питательных качеств существенно разнятся своим воздействием на пищеварение К примеру яблоки с морковью или с бананами полезны при склонности к диарее а абрикосы сливы с яблоками или груши напротив обладают послабляющим действием Необычные сочетания фруктов и ягод не просто уловка для маленьких гурманов микс помогает повысить пищевую ценность блюда обогатить его витаминами микроэлементами и растительными волокнами ', '1 ', 'абрикос']
['Варенье из абрикосов Абрикосы промойте в холодной воде сделайте надрезы выдавите косточки Косточки расколите ядра вставьте в отверстие абрикосов ', '1 ', 'абрикос']
['Варенье из абрикосов без косточек На 1 кг абрикосов 6 стаканов сахарного песка 5 стаканов воды Абрикосы разрезать вдоль бороздки удалить косточки ', '1 ', 'абрикос']
['Варенье из абрикосов Пятиминутка из абрикосов Компот из вишни ', '1 ', 'абрикос']
['Варенье из абрикосов с косточками На 1 кг абрикосов 7 стаканов сахарного песка 6 с

['В садах Узбекистана Туркмении и Таджикистана под абрикосом занято от 35 до 50 процентов площади Плоды абрикоса ценное сырье для консервной промышленности Однако консервные заводы часто получали это сырье один раз в несколько лет так как урожаи абрикоса отличаются большой нерегулярностью ', '2 ', 'абрикос']
['В самом деле Петрович позвоните и узнайте в чем там дело Ладно только торт шоколадный с абрикосами не весь съедайте Петрович идет к рабочему столу Главы государства поднимает трубку и набирает номер в то же время медведь оборачивается к окну и обращает свою морду вовне ', '1 ', 'абрикос']
['Всего доброго Я сегодня ела абрикосы и яблоки Вкусно ', '1 ', 'абрикос']
['Все интересовало каждый пустяк Антон Павлович любил ходить и показывать и рассказывать чего еще нет и что должно быть со временем и главное занимал его сад фруктовые посадки С помощью сестры Марии Павловны Антон Павлович сам рисует план сада намечает где будет какое дерево где скамеечка выписывает со всех концов России 

['Дома с пристройками балконами балкончиками и арочными галереями походили на маленькие старинные замки Пахло абрикосами и дымом от мангалов Рядом с домом Роберта раскинулось огромное развесистое дерево неизвестной Кириллу породы с дуплом аккуратно забитым куском жести ', '1 ', 'абрикос']
['Дом был у них большой комнат на двух этажах несчетно две кухни три спальни огромный зал с камином рядом озеро и богатый южный сад персики груши яблони абрикосы виноград И вдруг беда нахватавшемуся радиации Юрию стало и вовсе худо врачи велели ему лечь на операцию да только перед этим разжиться ', '1 ', 'абрикос']
['Дорога очень дурная и опасная она сначала обогнула гору Ульфет оставляя ее влево Прошли несколько деревень по долине речка сады и вообще растительность чудесные дуб тут орех груша яблоня абрикос кислые ароматные ягоды в роде дерена дикий виноград вьющийся по деревьям много незнакомых деревьев и кустов Лагерь в прекрасной небольшой долине на берегу речки Дарма кругом горы с лесом ширина ре

['и диалектическую Поэтому если подавались фрукты он абсолютно ел яблоки абсолютно не ел абрикосов апельсины ел в Петербурге не ел в провинции видите в Петербурге ', '1 ', 'абрикос']
['И еще одно чрезвычайно важное условие Любые попытки вырастить абрикосы можно предпринимать только в благоприятном микроклимате ', '2 ', 'абрикос']
['Известная немецкая косметическая фирма провела презентацию нового крема с экстрактом абрикоса и витамином С Это увлажняющее средство содержит светоотражающие пигменты придающие коже здоровый и естественный блеск Абрикос нареченный в народе плодом долгожителей насыщает кожу витаминным коктейлем смягчая и укрепляя ее Крем предназначен молодым женщинам до 35 лет и подходит для любой кожи ', '1 ', 'абрикос']
['изводится очень легко и канавки текучей воды пересекают почву во всех направлениях Кроме фиников довольно хорошего качества здесь в изобилии растут сладкие и горькие апельсины лучшие в Египте кислые и сладкие лимоны тутовые деревья в это время стоящие подо

...

['Особенно отрадно было видеть после Сунгачских болот лесистые горы и сухие долины одетые в самый пышный майский наряд В полном убранстве цветущих ландышей желтых лилий и других цветов красовалась живописная долина Сиян-хэ окаймленная лесистыми горами где среди разнообразнейших пород деревьев попадаются и абрикосы Желая хорошенько ознакомиться с страною Пржевальский шел обыкновенно напрямик куда глаза глядят ', '2 ', 'абрикос']
['Осторожно и мягко заговорила я с дочкой как наверное заговорила бы с душевнобольным человеком Я тебе привезла абрикосы груши виноград ты ведь любишь груши А где они с интересом спросила Настенька заглядывая в сумку ', '1 ', 'абрикос']
['Откуда тебе известно что я люблю именно этот сорт Про абрикосы не скажу ни слова Их и есть-то даже совестно ', '1 ', 'абрикос']
['Отмена крепостного права лишившая дворянское сословие даровой рабочей силы заставила покойного Сапегина заложить имение в банк для других новейших по моде века сельскохозяйственных начинаний они долж

['По случаю нового года и праздника весны эмиром бухарским устроено бесплатное народное гуляние к сожалению испорченное небывалым в это время морозом и снегом Цветы абрикоса персика и миндаля погибли неизвестный ', '1 ', 'абрикос']
['По соседству находилась учебная ферма для практических занятий Кроме того имелись теплицы и оранжереи где выращивали виноград персики абрикосы Основные доходы от имения владельцы должны были получать в виде оброка от крестьянских хозяйств ', '1 ', 'абрикос']
['Построен омшаник для зимовки пчел начата кирпичная кладка огромной оранжереи Ученики посадили множество груш слив яблонь длинные аллеи вишни-шпанки абрикосов сладкой мичуринской рябины и целые проспекты великолепных штамбовых роз И последнее дело о котором все говорят главная гордость Конотопа трамвай ', '1 ', 'абрикос']
['Потом Велька залезал вверх цепляясь за шершавый ствол и сначала что было сил тряс ветки руками а потом уже и азартно бил по ним ногой Абрикосы оранжевым градом летели вниз били в п

['Сад возле которого стоял санпоезд был сиротливо пуст но девушки нашего вагона сестра Клава и санитарка Аня были здешние кубанского рода и знали что до самой зимы до секучих зимних ветров на какой-нибудь ветке или дереве непременно задержится один-другой фрукт да и падалица бывает Они пошли в глубь сада и скоро вернулись оттуда неся в карманах и полах белых халатиков чуть порченные с боку в плесневелых лишаях абрикосы подопрелые яблоки и сказали что наберут груш У кого-то сыскался рюкзак кто-то изъявил желание пойти с девчатами в сад и скоро ', '1 ', 'абрикос']
['сад Когда мы пришли к нему в дом он потчевал нас прекрасными абрикосами и хорошим красным вином не купленным а домашним между тем дочь его играла приятно на клавесине Часов в семь поплыли мы назад в Цирих и я имел удовольствие видеть снежные горы позлащаемые заходящим солнцем и наконец помраченные густыми тенями вечера ', '1 ', 'абрикос']
['Сама же Зинаида Сергеевна оставшись рано вдовой с маленькой дочерью очень строго вдове

['Только Сталин почему-то прошептал но довольно слы шно Я этому Хрущеву когда встречу сам абрикосы отрежу Перечисление потерь и утрат не радует никого ', '2 ', 'абрикос']
['Только цвет у него будет особый золотистый Такую окраску Веторон должен приобрести благодаря бета-каротину природному веществу содержащемуся в облепихе шиповнике моркови абрикосах и многих других фруктах ягодах и овощах Говорят что бета-каротин необходим организму человека ', '1 ', 'абрикос']
['торговым путям и в монастырских садах Около 800 г н э культура абрикоса уже проникла в Германию и Северную Францию ', '2 ', 'абрикос']
['т п из наших абрикосов получаются превосходного качества В настоящее время в широкой продаже можно встретить сеянцы абрикоса Прежде чем ', '1 ', 'абрикос']
['Тридцать пять горящих свечей воткнутых в пышное тесто густо посыпанное сахарной пудрой обозначали число лет рожденницы Все дачники были приглашены под абрикосы к утреннему чаю День начавшийся так торжественно продолжался в том же духе и

['Это вам Петр Федорыч государыня изволили прислать в подарок Орлов открыл сундук и начал выгружать на стол снедь питие сладости жареный в сахаре миндаль глазированные померанцы абрикосы персики целые горы пряников закусок батареи английского портера бургундского картузы с ароматным табаком Ну чем бог послал ', '1 ', 'абрикос']
['Этого мне только не хватало Тут позвонил Абрикосов и спрашивает У тебя случайно нет моего папы ', '1 ', 'абрикос']
['Это заблуждение Есть такие фрукты персик абрикос груша из которых практически невозможно получить 100 процентов сока Поэтому из них делают фруктовое пюре ', '1 ', 'абрикос']
['Это к тетушке приходят соседки Они бойко обсуждают варенья и соленья смотрят на просвет банки с клубничным малиновым абрикосовым вишневым вареньем в саду растут и клубника и малина и абрикосы они смотрят на просвет сравнивая оттенки сиропа этот темноват переварено здесь сахару переложено засахарится они рассматривают пристально сощурив г лаза как алхимики как добыватели че

In [113]:
for p in Special_Dict:
    if 'Агония' in p:
        print(p['Агония'])
        print('Yes')


2
Yes


In [101]:
#Поиск всех ссылок в ссылках
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
from urllib.parse import unquote
from urllib.error import URLError, HTTPError
import re

def get_page_url(target):
    return "http://www.symbolarium.ru//index.php//{0}".format(quote(target))
try:
    SymbolFile = urllib.request.urlopen(get_page_url(unquote("Абрикос")))
except URLError as e:
    print('error')
else:
    SymbolHtml = SymbolFile.read()
    soup = BeautifulSoup(SymbolHtml)
    SymbolFile.close()
    print(soup.prettify())
    #print(soup.get_text())

        
        
print('Ready')      


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="ru">
 <head>
  <meta charset="utf-8"/>
  <title>
   Краткая энциклопедия символов
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Краткая_энциклопедия_символов","wgTitle":"Краткая энциклопедия символов","wgCurRevisionId":163181,"wgRevisionId":163181,"wgArticleId":42244,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Arb-служебные страницы"],"wgBreakFrames":false,"wgPageContentLanguage":"ru","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май",


Ready


In [25]:
for sentence in final[1]:
    print(sentence)

['1 900 65 1860 2 литровые банки 1 чайный стакан Для приготовления на зиму компотов более всего пригодны вишня черешня сливы абрикосы персики крыжовник смородина и клубника Плоды обмыть отцедить перебрать удалить веточки ', '1 ', 'rucontext\\абрикос.txt']
['1 Абрикосов Хрисанф Николаевич 1877 1957 единомышленник Л Н Толстого был женат на его внучатой племяннице Н Л Оболенской С осени 1902 г по 19 июня 1903 г подолгу жил в Ясной Поляне и помогал писателю в переписке его произведений и в ответах на письма ', '1 ', 'rucontext\\абрикос.txt']
['1 но не это было важно а важно приподнятое настроение которое мы застали в городе и в гостинице что к моему утешению выразилось между прочим в том что в каждом номере гостиницы стояла большая ваза полная бесплатных фруктов абрикосов персиков груш слив и винограда так что с голоду нельзя было умереть 1 ', '1 ', 'rucontext\\абрикос.txt']
['2004.04 06 20 14 в Хиппе есть абрикос с творогом наверное еще какие-то но мы только этот пробовали собираюсь набра

In [9]:
import glob
symbols=[]
for filename in glob.glob('rucsv\*.csv'):
    mypath=filename.replace('rucsv\\','')
    mypath=mypath.replace('.csv','')
    with open('RuSymbols\\'+mypath+'.txt','w',encoding='utf-8') as f:
        f.write('')
print('ready')

ready


In [10]:
#Временные символы
import glob
symbols=[]
for filename in glob.glob('RuSymbols\*.txt'):
    line=''
    filename=filename.replace('RuSymbols\\','')
    filename=filename.replace('.txt','')
    for word in filename.split('_'):
        line+=word+' '
    tokens = prepare_text_for_lda(line)
    for token in tokens:
        symbols.append(token)
print(symbols)

['абрикос', 'абрикосовый', 'абсолютный', 'абстрактный', 'абсурд', 'авангард', 'аванс', 'автомат', 'автономия', 'автономный', 'автор', 'авторитет', 'авторитетный', 'агент', 'агентство', 'агония', 'аграрный', 'агрессивный', 'агрессия', 'адвокат', 'адекватный', 'административный', 'администрация', 'адрес', 'адский', 'азбука', 'азиатский', 'академия', 'активный', 'акула', 'акцент', 'акционерный', 'акция', 'алгоритм', 'алкоголь', 'алкогольный', 'аллергия', 'алмаз', 'алмазный', 'алтарь', 'алчный', 'альбом', 'альянс', 'амбициозный', 'амбразура', 'американский', 'аморальный', 'анализ', 'ананас', 'анархия', 'анатомия', 'ангел', 'английский', 'анекдот', 'анкета', 'анонимный', 'ансамбль', 'антикварный', 'античный', 'антракт', 'апелляция', 'апельсин', 'апельсиновый', 'аппарат', 'арабский', 'арбуз', 'аргумент', 'арен', 'аренда', 'арест', 'аристократический', 'аристократия', 'арка', 'арктический', 'армия', 'армянский', 'аромат', 'ароматный', 'арсенал', 'артерия', 'артиллерия', 'артист', 'артистическ

In [41]:
#Стандарные символы
import glob
symbols_true=[]
for filename in glob.glob('Symbols\*.txt'):
    line=''
    filename=filename.replace('Symbols\\','')
    filename=filename.replace('.txt','')
    for word in filename.split('_'):
        line+=word+' '
    tokens = prepare_text_for_lda(line)
    for token in tokens:
        symbols_true.append(token)

In [40]:
# Одиночные вхождения имени, фамилии, отчества
from natasha import NamesExtractor

text=''''''
#with open('Lyrics\Chapter3.txt',encoding='utf-8') as f:
with open('Lyrics\Master.txt',encoding='utf-8') as f:
    for line in f:
        text+=line

Names=[]
Families=[]
Lasts=[]
extractor = NamesExtractor()
matches = extractor(text)
for match in matches:
    if match.fact.first is not None:
        tokens=prepare_text_for_lda(match.fact.first)
        if len(tokens)>0:
            Names.append(tokens[0])
    if match.fact.last is not None:
        tokens=prepare_text_for_lda(match.fact.last)
        if len(tokens)>0:
            Families.append(tokens[0])
    if match.fact.middle is not None:
        tokens=prepare_text_for_lda(match.fact.middle)
        if len(tokens)>0:
            Lasts.append(tokens[0])


Names_set=set(Names)
Fam_set=set(Families)
Lasts_set=set(Lasts)
print(Names_set)
print(Fam_set)
print(Lasts_set)

{'гамаленькая', 'кесария', 'жорж', 'иосиф', 'хеврон', 'клодин', 'андрей', 'адельфина', 'софья', 'джулль', 'ариман', 'пелагея', 'степан', 'фауст', 'мстислав', 'наташа', 'марья', 'алоизий', 'николай', 'савва', 'куролес', 'сергей', 'тузбубен', 'иванушка', 'дунчиль', 'амвросий', 'никанор', 'ксения', 'стёпа', 'аполлон', 'аркадий', 'мельпомена', 'карпов', 'анне', 'федя', 'левий', 'анна', 'матвей', 'луиз', 'тимофей', 'понтия', 'витя', 'воланд', 'анфис', 'владимир', 'павел', 'пантелей', 'григорий', 'гарун', 'панаева', 'степ', 'роман', 'дарья', 'венера', 'даниль', 'милица', 'антоний', 'фрид', 'прасковья', 'массолит', 'сашка', 'герберт', 'кедрон', 'гестас', 'маргарита', 'гессара', 'тузабубен', 'александр', 'фаланда', 'прохор', 'аристотель', 'иван', 'афраний', 'гамаленький', 'иммануил', 'михаил', 'драмлита', 'марк', 'семплеяр', 'ричард', 'максимилиан', 'рудольф', 'вульф', 'наталья', 'азазелло', 'евгений', 'антонид', 'иванов', 'василий', 'иисус', 'роберт', 'клавдий', 'малюта', 'валерий', 'иоганн',

In [42]:
custom_useless_words=['из-за','марка','ведь','патриарший','борменталь','филипп','кихот','санчо','антоний','сансон','шарик','мой','такой','другой','ваш','этот','сам','свой','тот','варенуха','степ','маргарита','иван','воланд','коровьев','пилат','николай','какой-то','свой','самый','быть','стать','это','что-то','весь','какой','мочь','никанор','который','ты','никакой','азазело','азазелло','берлиоз']

useless_set=set(custom_useless_words)
for name in Names_set:
    useless_set.add(name)
for name in Fam_set:
    useless_set.add(name)
for name in Lasts_set:
    useless_set.add(name)

In [78]:
def check_ngramm(ngramm,decision_value):
    count=0
    res=decision_value*len(ngramm)
    for gramm in ngramm:
        tokens=prepare_text_for_lda(gramm)
        if not(len(tokens)>0 and (tokens[0] not in useless_set)):
            continue
        for symbol in symbols_true:
            if tokens[0] in symbol:
                count+=1
                break
    if (count>=res)and len(ngramm)>1:
        return True
    else:
        return False

def get_label(string,word):
    words=prepare_text_for_lda(string)
    index=words.index(word)
    result=False
    ngramm=[]
    arg1=''
    arg2=''
    if not len(words)>2:
        return 0,'',''
    if index==0:
        ngramm.append(word)
        ngramm.append(words[1])
        for symbol in symbols_true:
            if words[1] in symbol:
                arg1=words[1]
                break
        ngramm.append(words[2])
        for symbol in symbols_true:
            if words[2] in symbol:
                arg2=words[2]
                break
        result=check_ngramm(ngramm,0.75)
    elif index==len(words)-1:
        ngramm.append(word)
        ngramm.append(words[index-1])
        for symbol in symbols_true:
            if words[index-1] in symbol:
                arg1=words[index-1]
                break
        ngramm.append(words[index-2])
        for symbol in symbols_true:
            if words[index-2] in symbol:
                arg2=words[index-2]
                break
        result=check_ngramm(ngramm,0.75)
    else:
        ngramm1=[]
        ngramm.append(words[index+1])
        ngramm.append(word)
        ngramm.append(words[index-1])
        arg1=words[index-1]
        arg2=words[index+1]
        result=(check_ngramm(ngramm,0.75) or check_ngramm(ngramm1,0.75))
    if result:
        return 1,arg1,arg2
    else:
        return 0,arg1,arg2

In [83]:
import csv
import pymorphy2
from docx import Document

separators=['...','?','!','.']
count_change=0
count_symbols_var=0
prev_value=False
ngramm=[]
results=[]
AddDict=[]
filename='Master'
with open('Lyrics\\'+filename+'.txt',encoding='utf-8') as f:
    document=Document()
    for line in f:
        prev_value=False
        if not len(line)>1:
            continue
        paragraph=document.add_paragraph('')
        stroks=line.split('...')
        temps=[]
        for strok in stroks:
            some=strok.split('?')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            some=strok.split('!')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            some=strok.split('.')
            for one in some:
                temps.append(one)
        stroks=temps
        temps=[]
        for strok in stroks:
            some=strok.replace('\n','')
            temps.append(some)
        stroks=temps
        for strok in stroks:
            tokens=prepare_text_for_lda(strok)
            for token in tokens:
                if token in symbols and {token:strok} not in results:
                    results.append({token:strok})

for result in results:
    keys=list(result.keys())
    key=keys[0]
    arg1=''
    arg2=''
    label=0
    if len(result[key].split())>2:
        label,arg1,arg2=get_label(result[key],key)
        if label==1:
            sentence=result[key].replace(';',',')
            AddDict.append({"arg1":arg1,"arg2":arg2,"word":key,"sentence":sentence,"label":label})

FILENAME = "rucsv\\"
filename='test_only_one'
with open(FILENAME+filename+'.csv', "w", newline="") as file:
    columns = ["arg1","arg2","word","sentence","label"]
    writer = csv.DictWriter(file, fieldnames=columns)
    writer.writeheader()
    for record in AddDict:
        try:
            writer.writerow(record)
        except UnicodeEncodeError as e:
            continue


In [81]:
for Add in AddDict:
    print(Add)

{'arg1': 'адский', 'arg2': 'голова', 'word': 'боль', 'sentence': ' Прокуратор был как каменный, потому что боялся качнуть пылающей адской болью головой', 'label': 1}
{'arg1': 'боль', 'arg2': 'адский', 'word': 'голова', 'sentence': ' Прокуратор был как каменный, потому что боялся качнуть пылающей адской болью головой', 'label': 1}
{'arg1': 'оставить', 'arg2': 'дворец', 'word': 'время', 'sentence': ' Я советовал бы тебе, игемон, оставить на время дворец и погулять пешком где-нибудь в окрестностях, ну хотя бы в садах на Елеонской горе', 'label': 1}
{'arg1': 'золотой', 'arg2': 'круглый', 'word': 'венец', 'sentence': ' На этой плешивой голове сидел редкозубый золотой венец; на лбу была круглая язва, разъедающая кожу и смазанная мазью; запавший беззубый рот с отвисшей нижней капризною губой', 'label': 1}
{'arg1': 'государственный', 'arg2': 'взгляд', 'word': 'власть', 'sentence': '–\xa0Да, – немного удивившись осведомленности прокуратора, продолжал Иешуа, – попросил меня высказать свой взгляд

{'arg1': 'красный', 'arg2': 'чаша', 'word': 'вино', 'sentence': ' Падая, он поддел ногой другую скамеечку, стоявшую перед ним, и с нее опрокинул себе на брюки полную чашу красного вина', 'label': 1}
{'arg1': 'место', 'arg2': 'живой', 'word': 'адский', 'sentence': ' Вот адское место для живого человека', 'label': 1}
{'arg1': 'украсть', 'arg2': 'гроб', 'word': 'голова', 'sentence': ' Граждане шептались о том, что у какого-то покойника, а какого – они не называли, сегодня утром из гроба украли голову', 'label': 1}
{'arg1': 'противоположный', 'arg2': 'река', 'word': 'берег', 'sentence': 'Маргарита же пронзительно свистнула и, оседлав подлетевшую щетку, перенеслась над рекой на противоположный берег', 'label': 1}
{'arg1': 'лестница', 'arg2': 'глаз', 'word': 'боль', 'sentence': ' Швейцарская и лестница, до боли в глазах залитая светом, были пусты', 'label': 1}
{'arg1': 'перо', 'arg2': 'туфля', 'word': 'голова', 'sentence': ' Теперь уже на каждой ступеньке оказались, издали казавшиеся соверше